In [32]:
%load_ext autoreload
%autoreload
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, InputLayer
from keras.layers.convolutional import Conv2D, MaxPooling2D
import sys
import os
pwd = os.getcwd()
sys.path.append("/root/Gan/jidian/MLexperiments")
sys.path.append("/root/Gan/jidian")
sys.path.append(pwd)
sys.path.append(os.path.dirname(os.getcwd()))
import MLexperiments.classes
from MLexperiments.classes import ReadAutoLabeledData
import tensorflow as tf
import MLexperiments.config.parameters
from keras.layers import LSTM, Reshape
import matplotlib.pyplot as plt
import utils
import keras
from keras.callbacks import ModelCheckpoint
import os
from keras.callbacks import TensorBoard
from time import time
from keras.models import Model
from keras.layers import Input, Dense


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4"
CHECKPOINTPATH = 'logs/{}/Rnn_weights.hdf5'.format(time())

FLAGS = None
myDataSet = utils.read_data_sets(one_hot=True, test_size = 0.1, validation_size = 0.1,fake_data = False)

(X_train, y_train), (X_test, y_test) , (X_Ftest, y_Ftest)= (myDataSet.train.images, myDataSet.train.labels), (myDataSet.validation.images, myDataSet.validation.labels), (myDataSet.test.images, myDataSet.test.labels)

print(X_train[0].shape)
print(y_train[0])

X_train = X_train.reshape(X_train.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT, 1).astype('float32')
X_Ftest = X_Ftest.reshape(X_Ftest.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT, 1).astype('float32')

#X_train /= 255
#X_test /= 255

def tran_y(y):
    y_ohe = np.zeros(2)
    if y:
        y_ohe[1] = 1
    else:
        y_ohe[0] = 1
    return y_ohe
# def tran_y(y):
#     return y

y_train_ohe = np.array([tran_y(y_train[i]) for i in range(len(y_train))])
y_test_ohe = np.array([tran_y(y_test[i]) for i in range(len(y_test))])




[autoreload of utils failed: Traceback (most recent call last):
  File "/home/anaconda3/envs/py3.5/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/home/anaconda3/envs/py3.5/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 369, in superreload
    module = reload(module)
  File "/home/anaconda3/envs/py3.5/lib/python3.5/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/anaconda3/envs/py3.5/lib/python3.5/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 626, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 780, in get_code
  File "<frozen importlib._bootstrap_external>", line 740, in source_to_code
  File "<frozen importlib._bootstrap>", line 222, in _call_with_frames_removed
  File "/root/Gan/jid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


KeyboardInterrupt: 

In [ ]:
import imp
imp.reload(utils)
imp.reload(MLexperiments.classes.DataSet)

In [ ]:
BATCH_SIZE=128
model = Sequential()
#model.add(Input(shape=(MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT, 1)))
# model.add(
#     Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1),input_shape=(MLexperiments.config.parameters.SAMPLE_LEN,\
#                                                                        MLexperiments.config.parameters.SAMPLE_HEIGHT, 1)\
#            , padding='valid', activation='relu',name="conv1"))

model.add(
    Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1),batch_input_shape=(None, MLexperiments.config.parameters.SAMPLE_LEN,\
                                                                       MLexperiments.config.parameters.SAMPLE_HEIGHT, 1)\
           , padding='valid', activation='relu',name="conv1"))

#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5,name="dropout1"))
model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu',name="conv2"))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5,name="dropout2"))
model.add(Conv2D(32, kernel_size=(10, MLexperiments.config.parameters.SAMPLE_HEIGHT - 4),name="conv3", strides=(1, 1), padding='valid', activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5,name="dropout3"))

model.add(Reshape((model.get_layer("dropout3").output_shape[1], model.get_layer("dropout3").output_shape[3]), name = "reshape"))

model.add(LSTM(128,  name = "LSTM1"))

model.add(Dense(64, activation='relu', name = "dense1"))
model.add(Dense(32, activation='relu', name = "dense2"))
model.add(Dense(2, activation='softmax', name = "dense3"))


In [ ]:
# BATCH_SIZE=128

# a = Input(shape=(MLexperiments.config.parameters.SAMPLE_LEN,\
#                                                                        MLexperiments.config.parameters.SAMPLE_HEIGHT, 1))
# #model.add(Input(shape=(MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT, 1)))
# # model.add(
# #     Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1),input_shape=(MLexperiments.config.parameters.SAMPLE_LEN,\
# #                                                                        MLexperiments.config.parameters.SAMPLE_HEIGHT, 1)\
# #            , padding='valid', activation='relu',name="conv1"))

# # x =    Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1)\
# #            , padding='valid', activation='relu',name="conv1")(a)
# x = Flatten()(a)

# x = Dense(2, activation='softmax', name = "dense3")(x)

# model = Model(inputs = [a], outputs = [x])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
lossHistory = LossHistory()
checkpointer = ModelCheckpoint(filepath=CHECKPOINTPATH, verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir="logs/{}".format(time()),histogram_freq=0, batch_size=BATCH_SIZE, write_graph=True, write_grads=False, \
                          write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

model.summary()
#



In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "8,7"

history = model.fit(X_train, y_train_ohe, validation_data=(X_test, y_test_ohe), epochs=10, batch_size=BATCH_SIZE, \
                callbacks=[\
                           lossHistory,\
                           checkpointer,\
                           tensorboard\
                          ] )

In [ ]:

# summarize history for accuracy
plt.plot(lossHistory.acc)
plt.plot(lossHistory.val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('batch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(lossHistory.losses)
plt.plot(lossHistory.val_losses)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('batch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

scores = model.evaluate(X_test, y_test_ohe, verbose=0)

In [ ]:
scores

In [ ]:

for index in range(10):
    print("expected: "+str( y_Ftest[index]))
    prediction = model.predict(np.expand_dims(X_Ftest[index], axis=0))[0]
    print("prediction:" + str(np.argmax(prediction,axis = 0)))
    time_series = X_Ftest.reshape(X_Ftest.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT) \
        .astype('float32')[index]
    # plt.imshow( X_train.reshape(X_train.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT).astype('float32')[0]);
    # plt.colorbar()
    # plt.show()
    plt.plot(time_series.T[0])
    plt.plot(time_series.T[1])
    plt.plot(time_series.T[2])
    plt.plot(time_series.T[3])
    plt.plot(time_series.T[4])
    plt.plot(time_series.T[5])
    plt.plot(time_series.T[6])
    plt.plot(time_series.T[7])
    plt.plot(time_series.T[8])
    plt.plot(time_series.T[9])
    plt.show()
   

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# datagen = ImageDataGenerator(
#        vertical_flip=False,
#         horizontal_flip=True)

# index = 0
# time_series = X_test.reshape(X_test.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT) \
#         .astype('float32')[index]

# # plt.plot(time_series.T[0])
# # plt.show()    
# # plt.plot(time_series.T[1])
# # plt.show()    
# # plt.plot(time_series.T[2])
# # plt.show()    
# # plt.plot(time_series.T[3])
# # plt.show()    
# # plt.plot(time_series.T[4])
# # plt.show()    
# # plt.plot(time_series.T[5])
# # plt.show()    
# # print("+++++++++++++++++++++++++++++++++++++++++++++")    
    
    
# # img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
# # x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
# # x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
# x = time_series.reshape((1,)+time_series.shape + (1,))
# print("x.shape  ")
# print(x.shape)
# # the .flow() command below generates batches of randomly transformed images
# # and saves the results to the `preview/` directory
# i = 0
# for batch in datagen.flow(x, batch_size=1):
#     i += 1
#     time_series = batch[0]
#     time_series = time_series.reshape( MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT) \
#         .astype('float32')
#     plt.plot(time_series.T[0])
#     plt.plot(time_series.T[1])
#     plt.plot(time_series.T[2])
#     plt.plot(time_series.T[3])
#     plt.plot(time_series.T[4])
#     plt.plot(time_series.T[5])
#     plt.show()
# #     plt.plot(time_series.T[0])
# #     plt.show()    
# #     plt.plot(time_series.T[1])
# #     plt.show()    
# #     plt.plot(time_series.T[2])
# #     plt.show()    
# #     plt.plot(time_series.T[3])
# #     plt.show()    
# #     plt.plot(time_series.T[4])
# #     plt.show()    
# #     plt.plot(time_series.T[5])
# #     plt.show()    
# #     print("+++++++++++++++++++++++++++++++++++++++++++++")    
    
#     if i > 3:
#         break  # otherwise the generator would loop indefinitely